In [1]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd

In [2]:
df = pd.read_csv('../data/df_VEH0145.csv')

C:\Users\Paul\AppData\Local\Temp\ipykernel_15980\3724977010.py:1: DtypeWarning: Columns (45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/df_VEH0145.csv')


In [3]:
df.replace('[c]', np.nan, inplace=True)

In [4]:
df2 = pd.melt(df, id_vars=['LSOA11CD', 'LSOA11NM', 'Fuel', 'Keepership'])

In [5]:
df2.rename(columns={'variable': 'Period', 'value': 'Vehicles'}, inplace=True)

In [6]:
df2 = df2[(df2['Fuel']!= 'Total') & (df2['Keepership']!= 'Total')]

In [7]:
df2

,LSOA11CD,LSOA11NM,Fuel,Keepership,Period,Vehicles
0,95AA01S1,Aldergrove 1,Battery electric,Company,2022 Q3,0
1,95AA01S2,Aldergrove 2,Battery electric,Company,2022 Q3,NaN
2,95AA01S3,Aldergrove 3,Battery electric,Company,2022 Q3,NaN
3,95AA02W1,Balloo,Battery electric,Company,2022 Q3,NaN
4,95AA04W1,Clady,Battery electric,Company,2022 Q3,NaN
...,...,...,...,...,...,...
13284403,W01001933,Gwynedd 017F,Range extended electric,Private,2011 Q4,0
13284404,W01001934,Ceredigion 002F,Range extended electric,Private,2011 Q4,0
13284405,W01001948,Cardiff 006E,Range extended electric,Private,2011 Q4,0
13284406,W01001950,Cardiff 049D,Range extended electric,Private,2011 Q4,0


In [8]:
df2['Vehicles'] = df2['Vehicles'].astype(float)

In [9]:
fuel_time = df2.groupby(['Fuel', 'Period'])[['Vehicles']].sum().astype(int).reset_index()

In [10]:
fuel_time

,Fuel,Period,Vehicles
0,Battery electric,2011 Q4,4701
1,Battery electric,2012 Q1,4882
2,Battery electric,2012 Q2,5156
3,Battery electric,2012 Q3,5116
4,Battery electric,2012 Q4,5540
...,...,...,...
171,Range extended electric,2021 Q3,3984
172,Range extended electric,2021 Q4,4426
173,Range extended electric,2022 Q1,4586
174,Range extended electric,2022 Q2,4908


In [11]:
qs = fuel_time['Period'].str.replace(r' ', r'-')

In [12]:
fuel_time['Date'] = pd.PeriodIndex(qs, freq='Q').to_timestamp() + pd.offsets.QuarterEnd()

In [13]:
fuel_time

,Fuel,Period,Vehicles,Date
0,Battery electric,2011 Q4,4701,2011-12-31
1,Battery electric,2012 Q1,4882,2012-03-31
2,Battery electric,2012 Q2,5156,2012-06-30
3,Battery electric,2012 Q3,5116,2012-09-30
4,Battery electric,2012 Q4,5540,2012-12-31
...,...,...,...,...
171,Range extended electric,2021 Q3,3984,2021-09-30
172,Range extended electric,2021 Q4,4426,2021-12-31
173,Range extended electric,2022 Q1,4586,2022-03-31
174,Range extended electric,2022 Q2,4908,2022-06-30


In [14]:
fuel_time['Fuel'].value_counts()

Battery electric                    44
Plug-in hybrid electric (diesel)    44
Plug-in hybrid electric (petrol)    44
Range extended electric             44
Name: Fuel, dtype: int64

In [15]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')
'''
selectors = alt.Chart(source).mark_point().encode(
    x='x:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)
'''

"\nselectors = alt.Chart(source).mark_point().encode(\n    x='x:Q',\n    opacity=alt.value(0),\n).add_selection(\n    nearest\n)\n"

In [22]:
chart = alt.Chart(fuel_time, title="UK electric vehicles over time by type").mark_line().encode(
alt.X('Date:T'),
alt.Y('Vehicles:Q'),
alt.Color('Fuel'),
alt.Tooltip(['Vehicles', 'Date'])
).configure_title(anchor="middle", fontSize=16, dy=-16).add_selection(
    nearest
)

In [23]:
chart

alt.Chart(...)

In [24]:
chart.save('../charts/VEH0145_electric_type_time.json')